In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-aug-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-aug-2022/train.csv
/kaggle/input/tabular-playground-series-aug-2022/test.csv


In [2]:
train_file='/kaggle/input/tabular-playground-series-aug-2022/train.csv'
test_file='/kaggle/input/tabular-playground-series-aug-2022/test.csv'


In [3]:
train_df=pd.read_csv(train_file)
test_df=pd.read_csv(test_file)

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_squared_log_error
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
from imblearn.pipeline import Pipeline as imbpipeline

In [5]:
X = train_df.copy()
y = X.pop("failure")

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype == "object"]

numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [6]:
X_test_1 = test_df.copy()
# y = X.pop("failure")

# X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
#                                                                 random_state=0)

categorical_cols_test = [cname for cname in X_test_1.columns if X_test_1[cname].dtype == "object"]

numerical_cols_test = [cname for cname in X_test_1.columns if X_test_1[cname].dtype in ['int64', 'float64']]

my_cols_test = categorical_cols_test + numerical_cols_test
X_train_test = X_test_1[my_cols_test].copy()


In [7]:
numerical_transformer = Pipeline(steps=[
    ('imputer1',SimpleImputer(strategy='median')),
    ('scaler', StandardScaler(with_mean=False)),
#     ('poly_features', PolynomialFeatures(degree=2, include_bias=False)),
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer2', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ('scaler', StandardScaler(with_mean=False)),
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [8]:
X_train_tr = preprocessor.fit_transform(X_train, y_train)
X_valid_tr = preprocessor.transform(X_valid)

In [9]:
X_test_1=preprocessor.transform(X_train_test)

In [10]:
rf_model=RandomForestRegressor(n_estimators=160,n_jobs=-1)
model_rf=rf_model.fit(X_train_tr,y_train)

In [11]:
y_preds=model_rf.predict(X_valid_tr)

In [12]:
mean_squared_error(y_preds,y_valid)

0.1673337307701355

In [13]:
y_preds_test=model_rf.predict(X_test_1)

In [14]:
test_df['failure']=y_preds_test

In [15]:
test_df[['id','failure']].to_csv('submission.csv',index=False)

In [16]:
!head submission.csv

id,failure
26570,0.25625
26571,0.25
26572,0.325
26573,0.26875
26574,0.3375
26575,0.3375
26576,0.175
26577,0.25625
26578,0.14375
